In [ ]:
from gurobipy import GRB
from sklearn.model_selection import KFold

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 81.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum, Env
from sklearn.model_selection import KFold
from scipy.stats import laplace
from sklearn.metrics import mean_absolute_error

# **Import data and select features**

In [ ]:
df = pd.read_csv('StudentPerformanceFactors.csv')
selected_features = [
    'Hours_Studied',
    'Sleep_Hours',
    'Previous_Scores',
    'Family_Income',
    'Attendance',
    'Distance_from_Home'
]

df = df[selected_features + ['Exam_Score']].dropna()
print("Processed data shape:", df.shape)
X = df[selected_features]
y = df['Exam_Score']
feature_names = selected_features
print("Selected features:", feature_names)
X = df[selected_features]
y = df['Exam_Score']


income_mapping = {"Low": 0, "Medium": 1, "High": 2}
distance_mapping = {"Near": 0, "Moderate": 1, "Far": 2}

X["Family_Income"] = X["Family_Income"].map(income_mapping)
X["Distance_from_Home"] = X["Distance_from_Home"].map(distance_mapping)

Processed data shape: (6540, 7)
Selected features: ['Hours_Studied', 'Sleep_Hours', 'Previous_Scores', 'Family_Income', 'Attendance', 'Distance_from_Home']


<ipython-input-5-3110525892>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Family_Income"] = X["Family_Income"].map(income_mapping)
<ipython-input-5-3110525892>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Distance_from_Home"] = X["Distance_from_Home"].map(distance_mapping)


# **Part 1**

In [ ]:
#Split the data
kf = KFold(n_splits=5)
mae_scores = []

all_y_true = []
all_y_pred = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nProcessing Fold {fold+1}/5")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    knots = {}

    #Define five quantile positions
    quantiles = [0.10, 0.25, 0.5, 0.75, 0.9]

    #Get quantiles value
    for feature in feature_names:
        knots[feature] = [
            np.percentile(X_train[feature], q * 100)
            for q in quantiles
        ]
    #Get hingle value through X - knot_value
    def create_hinge_features(X_data, knots):
      X_hinge = X_data.copy()

      for feature in feature_names:
        for i, q in enumerate(quantiles):
          knot_val = knots[feature][i]
          hinge_name = f"{feature}_hinge_q{int(q*100)}"
          X_hinge[hinge_name] = np.maximum(0, X_data[feature] - knot_val)

      X_hinge['intercept'] = 1
      return X_hinge

    X_train_hinge = create_hinge_features(X_train, knots)
    X_test_hinge = create_hinge_features(X_test, knots)

    n_samples = len(X_train_hinge)
    n_features = len(X_train_hinge.columns)
    #Setup Guroipy environment
    env = Env(empty=True)
    env.setParam("WLSACCESSID", "af427699-3d84-4be7-8375-848e1c01fe5d")
    env.setParam("WLSSECRET", "660f7b8c-a673-4870-94ae-c0ad37c28454")
    env.setParam("LICENSEID", 2678165)
    env.start()


    model = Model("SplineRegression", env = env)
    model.setParam('OutputFlag',0)

    #Add variable
    beta = model.addVars(n_features, name = "beta")
    pred_y = model.addVars(n_samples, name = "pred_y")
    abs_error = model.addVars(n_samples, name = "abs_error")
    #Minimize MAE through objective function and constraint
    model.setObjective(quicksum(abs_error[i] for i in range(n_samples)), GRB.MINIMIZE)

    for i in range(n_samples):
      model.addConstr(
          pred_y[i] == quicksum(
              beta[j] * X_train_hinge.iloc[i, j]
              for j in range(n_features)
          )
      )
      model.addConstr(abs_error[i] >= y_train.iloc[i] - pred_y[i])
      model.addConstr(abs_error[i] >= pred_y[i] - y_train.iloc[i])

    model.optimize()

    if model.status == GRB.OPTIMAL:
      print("Optimal solution found")
    else:
      print("No optimal solution found")
      continue

    coefficients = np.array([beta[j].X for j in range(n_features)])
    X_test_mat = X_test_hinge.values
    y_pred = X_test_mat @ coefficients

    all_y_true.extend(y_test.values)
    all_y_pred.extend(y_pred)

    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)
    print(f"Fold {fold+1} MAE: {mae}")
cv_mae = np.mean(mae_scores)
print("\n===== Final Results =====")
print(f"5-fold CV MAE: {cv_mae:.2f}")
print("Fold MAEs:", [f"{score:.2f}" for score in mae_scores])


Processing Fold 1/5
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw
Optimal solution found
Fold 1 MAE: 1.419703345743124

Processing Fold 2/5
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw
Optimal solution found
Fold 2 MAE: 1.4270495120872824

Processing Fold 3/5
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw
Optimal solution found
Fold 3 MAE: 1.3587615108416016

Processing Fold 4/5
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw
Optimal solution found
Fold

In [ ]:
print("\n--- Spline Function Coefficients ---")
coef_idx = 0
for feature in feature_names:
    print(f"\nFeature: {feature}")
    print(f"  Intervals (hinge knots): {knots[feature]}")
    for i, q in enumerate(quantiles):
        knot = knots[feature][i]
        coef = coefficients[coef_idx]
        print(f"    Hinge @ q={int(q*100)} (x > {knot:.2f}): β = {coef:.4f}")
        coef_idx += 1

print(f"\nIntercept: β = {coefficients[coef_idx]:.4f}")



--- Spline Function Coefficients ---

Feature: Hours_Studied
  Intervals (hinge knots): [np.float64(12.0), np.float64(16.0), np.float64(20.0), np.float64(24.0), np.float64(28.0)]
    Hinge @ q=10 (x > 12.00): β = 0.2953
    Hinge @ q=25 (x > 16.00): β = 0.0000
    Hinge @ q=50 (x > 20.00): β = 0.0000
    Hinge @ q=75 (x > 24.00): β = 0.0000
    Hinge @ q=90 (x > 28.00): β = 0.1995

Feature: Sleep_Hours
  Intervals (hinge knots): [np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0)]
    Hinge @ q=10 (x > 5.00): β = 0.0000
    Hinge @ q=25 (x > 6.00): β = 0.0000
    Hinge @ q=50 (x > 7.00): β = 0.0000
    Hinge @ q=75 (x > 8.00): β = 0.0000
    Hinge @ q=90 (x > 9.00): β = 0.0140

Feature: Previous_Scores
  Intervals (hinge knots): [np.float64(55.0), np.float64(63.0), np.float64(75.0), np.float64(87.0), np.float64(95.0)]
    Hinge @ q=10 (x > 55.00): β = 0.0039
    Hinge @ q=25 (x > 63.00): β = 0.0000
    Hinge @ q=50 (x > 75.00): β = 0.0000
    Hinge @ q

# **Part 2**

In [ ]:
# Part 2: L1 Budgeted Spline Regression & Model Comparison

# Define L1 Budget Levels
budget_levels = [1,3,5,7,10,20,30,40,50,70,100]
print("Budget levels (Bi):", budget_levels)


budgeted_cv_results = {}

# Setup Gurobi environment
env = Env(empty=True)
env.setParam("WLSACCESSID", "af427699-3d84-4be7-8375-848e1c01fe5d")
env.setParam("WLSSECRET", "660f7b8c-a673-4870-94ae-c0ad37c28454")
env.setParam("LICENSEID", 2678165)
env.start()

try:
    for budget in budget_levels:
        print(f"\n{'='*50}")
        print(f"Processing Budget Level: {budget}")
        print(f"{'='*50}")

        kf = KFold(n_splits=5)
        mae_scores_budget = []

        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            print(f"\nFold {fold+1}/5 for Budget {budget}")

            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            knots = {}
            quantiles = [0.10, 0.25, 0.5, 0.75, 0.9]

            for feature in feature_names:
                knots[feature] = [
                    np.percentile(X_train[feature], q * 100)
                    for q in quantiles
                ]

            def create_hinge_features(X_data, knots):
                X_hinge = X_data.copy()

                for feature in feature_names:
                    for i, q in enumerate(quantiles):
                        knot_val = knots[feature][i]
                        hinge_name = f"{feature}_hinge_q{int(q*100)}"
                        X_hinge[hinge_name] = np.maximum(0, X_data[feature] - knot_val)

                X_hinge['intercept'] = 1
                return X_hinge

            # Create hinge features
            X_train_hinge = create_hinge_features(X_train, knots)
            X_test_hinge = create_hinge_features(X_test, knots)

            n_samples = len(X_train_hinge)
            n_features = len(X_train_hinge.columns)

            # Create Gurboi optimization model with L1 budget constraint
            model = Model(f"BudgetedSplineRegression_B{budget}", env=env)
            model.setParam('OutputFlag', 0)

            # Define variables
            beta = model.addVars(n_features, lb=-GRB.INFINITY, name="beta")
            pred_y = model.addVars(n_samples, name="pred_y")
            abs_error = model.addVars(n_samples, name="abs_error")

            # Auxiliary variables for L1 norm constraint
            beta_pos = model.addVars(n_features, name="beta_pos")
            beta_neg = model.addVars(n_features, name="beta_neg")

            # Set objective function (minimize MAE)
            model.setObjective(quicksum(abs_error[i] for i in range(n_samples)), GRB.MINIMIZE)

            # Prediction constraints
            for i in range(n_samples):
                model.addConstr(
                    pred_y[i] == quicksum(
                        beta[j] * X_train_hinge.iloc[i, j]
                        for j in range(n_features)
                    )
                )
                model.addConstr(abs_error[i] >= y_train.iloc[i] - pred_y[i])
                model.addConstr(abs_error[i] >= pred_y[i] - y_train.iloc[i])

            # L1 budget constraint: sum(|beta_j|) <= Budget
            for j in range(n_features):
                model.addConstr(beta[j] == beta_pos[j] - beta_neg[j])

            model.addConstr(
                quicksum(beta_pos[j] + beta_neg[j] for j in range(n_features)) <= budget
            )

            model.optimize()

            if model.status == GRB.OPTIMAL:
                coefficients = np.array([beta[j].X for j in range(n_features)])
                X_test_mat = X_test_hinge.values
                y_pred = X_test_mat @ coefficients
                mae = mean_absolute_error(y_test, y_pred)
                mae_scores_budget.append(mae)
                print(f"  Fold {fold+1} MAE: {mae:.4f}")

                if fold == 0:
                    budgeted_cv_results[budget] = {
                        'coefficients': coefficients,
                        'feature_names': list(X_train_hinge.columns),
                        'l1_norm': np.sum(np.abs(coefficients))
                    }
            else:
                print(f"  Fold {fold+1}: No optimal solution found")
                mae_scores_budget.append(np.inf)

            model.dispose()
        #Avoid invalid values
        valid_scores = [score for score in mae_scores_budget if score != np.inf]
        if valid_scores:
            cv_mae_budget = np.mean(valid_scores)
            budgeted_cv_results[budget]['cv_mae'] = cv_mae_budget
            budgeted_cv_results[budget]['fold_maes'] = valid_scores
            print(f"Budget {budget} CV-MAE: {cv_mae_budget:.4f}")
        else:
            budgeted_cv_results[budget] = {'cv_mae': np.inf}
            print(f"Budget {budget}: All folds failed")

finally:
    env.dispose()

# Model Comparison

print(f"\n{'='*60}")
print("MODEL COMPARISON RESULTS")
print(f"{'='*60}")

print("\nCV-MAE for different budget levels:")
print("-" * 40)
for budget in budget_levels:
    if budget in budgeted_cv_results:
        cv_mae = budgeted_cv_results[budget]['cv_mae']
        if cv_mae != np.inf:
            print(f"Budget {budget:6.1f}: CV-MAE = {cv_mae:.4f}")
        else:
            print(f"Budget {budget:6.1f}: Failed to converge")

valid_budgets = {b: results['cv_mae'] for b, results in budgeted_cv_results.items()
                 if results['cv_mae'] != np.inf}

if valid_budgets:
    optimal_budget = min(valid_budgets, key=valid_budgets.get)
    optimal_cv_mae = valid_budgets[optimal_budget]

    print(f"\nOptimal Budget B* = {optimal_budget}")
    print(f"Optimal CV-MAE = {optimal_cv_mae:.4f}")

    print(f"\nMODEL COMPARISON:")
    print(f"Unconstrained Model CV-MAE: 1.38")  # From Part 1
    print(f"Budgeted Model CV-MAE:     {optimal_cv_mae:.4f}")
    print(f"Improvement: {((1.38 - optimal_cv_mae) / 1.38 * 100):+.2f}%")

    if optimal_budget in budgeted_cv_results:
        optimal_results = budgeted_cv_results[optimal_budget]
        optimal_coeffs = optimal_results['coefficients']
        feature_names_hinge = optimal_results['feature_names']

        print(f"\nOPTIMAL MODEL ANALYSIS (B* = {optimal_budget}):")
        print("-" * 50)

        nonzero_coeffs = np.sum(np.abs(optimal_coeffs) > 0.001)
        print(f"Number of nonzero coefficients (|\u00b7| > 0.001): {nonzero_coeffs}")

        l1_norm = np.sum(np.abs(optimal_coeffs))
        print(f"L1 norm of coefficients: {l1_norm:.4f}")

        print(f"\nSpline coefficient analysis (\u03b3 only):")
        gamma_coeffs = []
        for coeff, name in zip(optimal_coeffs, feature_names_hinge):
            if 'hinge' in name:
                gamma_coeffs.append(abs(coeff))
        #Get gamma coefficients result and ranking
        if gamma_coeffs:
            avg_gamma_coeff = np.mean(gamma_coeffs)
            max_gamma_coeff = np.max(gamma_coeffs)
            active_gamma_count = np.sum(np.array(gamma_coeffs) > 0.001)

            print(f"  Average |\u03b3|: {avg_gamma_coeff:.4f}")
            print(f"  Maximum |\u03b3|: {max_gamma_coeff:.4f}")
            print(f"  Number of active \u03b3 terms: {active_gamma_count}")

            print(f"\nTop 10 \u03b3 coefficients by magnitude:")
            gamma_analysis = [
                (abs(coeff), name, coeff)
                for coeff, name in zip(optimal_coeffs, feature_names_hinge)
                if 'hinge' in name and abs(coeff) > 0.001
            ]
            gamma_analysis.sort(reverse=True)

            for i, (abs_coeff, name, coeff) in enumerate(gamma_analysis[:10]):
                print(f"  {i+1:2d}. {name:25s}: {coeff:8.4f} (|{abs_coeff:.4f}|)")
        else:
            print("  No active \u03b3 coefficients found.")
else:
    print("\nNo valid budget levels found. All models failed to converge.")
    print("Consider using larger budget values or checking the data.")

Budget levels (Bi): [1, 3, 5, 7, 10, 20, 30, 40, 50, 70, 100]
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw

Processing Budget Level: 1

Fold 1/5 for Budget 1
  Fold 1 MAE: 4.1787

Fold 2/5 for Budget 1
  Fold 2 MAE: 4.3248

Fold 3/5 for Budget 1
  Fold 3 MAE: 4.2766

Fold 4/5 for Budget 1
  Fold 4 MAE: 4.3413

Fold 5/5 for Budget 1
  Fold 5 MAE: 4.1085
Budget 1 CV-MAE: 4.2460

Processing Budget Level: 3

Fold 1/5 for Budget 3
  Fold 1 MAE: 1.4336

Fold 2/5 for Budget 3
  Fold 2 MAE: 1.4251

Fold 3/5 for Budget 3
  Fold 3 MAE: 1.3531

Fold 4/5 for Budget 3
  Fold 4 MAE: 1.3563

Fold 5/5 for Budget 3
  Fold 5 MAE: 1.2839
Budget 3 CV-MAE: 1.3704

Processing Budget Level: 5

Fold 1/5 for Budget 5
  Fold 1 MAE: 1.4330

Fold 2/5 for Budget 5
  Fold 2 MAE: 1.4198

Fold 3/5 for Budget 5
  Fold 3 MAE: 1.3520

Fold 4/5 for Budget 5
  Fold 4 MAE: 1.3529

Fold 5/

In [ ]:
print(f"{'='*60}")
print("PART 3: PREDICTION INTERVAL CONSTRUCTION & EVALUATION")
print(f"{'='*60}")

# Use optimal budget from Part 2
# You should replace this with the actual optimal budget found in Part 2
optimal_budget = optimal_budget  # From Part 2 results

print(f"Using optimal budget B* = {optimal_budget}")

# Initialize results storage
coverage_rates = []
msis_scores = []
all_intervals = []
all_true_values = []

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Setup Gurobi environment
env = Env(empty=True)
env.setParam("WLSACCESSID", "af427699-3d84-4be7-8375-848e1c01fe5d")
env.setParam("WLSSECRET", "660f7b8c-a673-4870-94ae-c0ad37c28454")
env.setParam("LICENSEID", 2678165)
env.start()

try:
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n{'='*40}")
        print(f"Processing Fold {fold+1}/5")
        print(f"{'='*40}")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Calculate knots (same as previous parts)
        knots = {}
        quantiles = [0.10, 0.25, 0.5, 0.75, 0.9]

        for feature in feature_names:
            knots[feature] = [
                np.percentile(X_train[feature], q * 100)
                for q in quantiles
            ]

        def create_hinge_features(X_data, knots):
            X_hinge = X_data.copy()

            for feature in feature_names:
                for i, q in enumerate(quantiles):
                    knot_val = knots[feature][i]
                    hinge_name = f"{feature}_hinge_q{int(q*100)}"
                    X_hinge[hinge_name] = np.maximum(0, X_data[feature] - knot_val)

            X_hinge['intercept'] = 1
            return X_hinge

        # Create hinge features
        X_train_hinge = create_hinge_features(X_train, knots)
        X_test_hinge = create_hinge_features(X_test, knots)

        n_samples = len(X_train_hinge)
        n_features = len(X_train_hinge.columns)

        # Fit budgeted model (same as Part 2)
        model = Model(f"BudgetedModel_Fold{fold+1}", env=env)
        model.setParam('OutputFlag', 0)

        # Define variables
        beta = model.addVars(n_features, lb=-GRB.INFINITY, name="beta")
        pred_y = model.addVars(n_samples, name="pred_y")
        abs_error = model.addVars(n_samples, name="abs_error")
        beta_pos = model.addVars(n_features, name="beta_pos")
        beta_neg = model.addVars(n_features, name="beta_neg")

        # Set objective
        model.setObjective(quicksum(abs_error[i] for i in range(n_samples)), GRB.MINIMIZE)

        # Add constraints
        for i in range(n_samples):
            model.addConstr(
                pred_y[i] == quicksum(
                    beta[j] * X_train_hinge.iloc[i, j]
                    for j in range(n_features)
                )
            )
            model.addConstr(abs_error[i] >= y_train.iloc[i] - pred_y[i])
            model.addConstr(abs_error[i] >= pred_y[i] - y_train.iloc[i])

        # L1 budget constraint
        for j in range(n_features):
            model.addConstr(beta[j] == beta_pos[j] - beta_neg[j])

        model.addConstr(
            quicksum(beta_pos[j] + beta_neg[j] for j in range(n_features)) <= optimal_budget
        )

        # Optimize
        model.optimize()

        if model.status == GRB.OPTIMAL:
            # Get coefficients
            coefficients = np.array([beta[j].X for j in range(n_features)])

            # 1. Residual Variance Modeling
            print("Step 1: Computing residuals and variance modeling...")

            # Compute residuals on training set
            X_train_mat = X_train_hinge.values
            y_train_pred = X_train_mat @ coefficients
            residuals = y_train.values - y_train_pred

            # Regress ln(r_i^2) on same spline features to estimate σ²(x)
            log_residuals_sq = np.log(np.maximum(residuals**2, 1e-8))  # Avoid log(0)

            # Fit variance model using OLS (since it's for variance estimation)
            from sklearn.linear_model import LinearRegression
            variance_model = LinearRegression()
            variance_model.fit(X_train_hinge.values, log_residuals_sq)

            # 2. Laplace-Based Interval Simulation
            print("Step 2: Laplace-based interval simulation...")

            # Predict on test set
            X_test_mat = X_test_hinge.values
            y_test_pred = X_test_mat @ coefficients

            # Estimate variance for test points
            log_var_pred = variance_model.predict(X_test_mat)
            sigma_i_sq = np.exp(log_var_pred)

            # For each test point
            fold_intervals = []
            fold_coverage = []
            fold_msis = []

            for i in range(len(X_test)):
                # Compute ŷᵢ and σ̂ᵢ²
                y_pred_i = y_test_pred[i]
                sigma_sq_i = sigma_i_sq[i]

                # Set bᵢ = √(σ̂ᵢ²/2)
                b_i = np.sqrt(sigma_sq_i / 2)

                # Draw S = 100 errors from Laplace(0, bᵢ)
                S = 100
                errors = laplace.rvs(loc=0, scale=b_i, size=S)

                # Simulate y^(s) = ŷᵢ + e^(s)
                y_simulated = y_pred_i + errors

                # Interval [Lᵢ, Uᵢ]: 5th & 95th percentiles
                L_i = np.percentile(y_simulated, 5)
                U_i = np.percentile(y_simulated, 95)

                fold_intervals.append([L_i, U_i])

                # Check coverage
                y_true_i = y_test.iloc[i]
                is_covered = (L_i <= y_true_i <= U_i)
                fold_coverage.append(is_covered)

                # Calculate MSIS (α = 0.10 for 90% interval)
                alpha = 0.10
                width = U_i - L_i
                lower_penalty = 2 * max(0, L_i - y_true_i) / alpha
                upper_penalty = 2 * max(0, y_true_i - U_i) / alpha

                msis_i = (width + lower_penalty + upper_penalty) / y_pred_i if y_pred_i != 0 else width + lower_penalty + upper_penalty
                fold_msis.append(msis_i)

            # Store fold results
            all_intervals.extend(fold_intervals)
            all_true_values.extend(y_test.values)

            fold_coverage_rate = np.mean(fold_coverage)
            fold_msis_mean = np.mean(fold_msis)

            coverage_rates.append(fold_coverage_rate)
            msis_scores.append(fold_msis_mean)

            print(f"Fold {fold+1} Coverage Rate: {fold_coverage_rate:.4f}")
            print(f"Fold {fold+1} Mean MSIS: {fold_msis_mean:.4f}")

        else:
            print(f"Fold {fold+1}: Optimization failed")
            coverage_rates.append(0.0)
            msis_scores.append(np.inf)

        model.dispose()

finally:
    env.dispose()

# 3. Evaluation Metrics
print(f"\n{'='*50}")
print("FINAL EVALUATION RESULTS")
print(f"{'='*50}")

# Overall coverage rate
overall_coverage = np.mean(coverage_rates)
overall_msis = np.mean([score for score in msis_scores if score != np.inf])

print(f"\nPrediction Interval Performance:")
print("-" * 40)
print(f"Target Coverage (90%):     0.9000")
print(f"Achieved Coverage Rate:    {overall_coverage:.4f}")
print(f"Coverage Difference:       {(overall_coverage - 0.9):+.4f}")

print(f"\nMean Scaled Interval Score (MSIS):")
print(f"Overall MSIS:              {overall_msis:.4f}")

print(f"\nPer-Fold Results:")
print("-" * 30)
for i in range(len(coverage_rates)):
    if msis_scores[i] != np.inf:
        print(f"Fold {i+1}: Coverage = {coverage_rates[i]:.4f}, MSIS = {msis_scores[i]:.4f}")
    else:
        print(f"Fold {i+1}: Failed")

# Additional Analysis
if len(all_intervals) > 0:
    all_intervals = np.array(all_intervals)
    interval_widths = all_intervals[:, 1] - all_intervals[:, 0]

    print(f"\nInterval Width Statistics:")
    print("-" * 30)
    print(f"Mean Width:                {np.mean(interval_widths):.4f}")
    print(f"Median Width:              {np.median(interval_widths):.4f}")
    print(f"Std Width:                 {np.std(interval_widths):.4f}")
    print(f"Min Width:                 {np.min(interval_widths):.4f}")
    print(f"Max Width:                 {np.max(interval_widths):.4f}")

# Quality Assessment
print(f"\nInterval Quality Assessment:")
print("-" * 35)
if overall_coverage >= 0.85 and overall_coverage <= 0.95:
    coverage_quality = "Good"
elif overall_coverage >= 0.80 and overall_coverage <= 0.98:
    coverage_quality = "Acceptable"
else:
    coverage_quality = "Poor"

print(f"Coverage Quality:          {coverage_quality}")
print(f"MSIS Performance:          {'Good' if overall_msis < 2.0 else 'Acceptable' if overall_msis < 5.0 else 'Poor'}")

print(f"\n{'='*60}")
print("PART 3 COMPLETED")
print(f"{'='*60}")

# Summary for all parts
print(f"\n{'='*60}")
print("COMPLETE PROJECT SUMMARY")
print(f"{'='*60}")
print(f"Part 1 - Unconstrained CV-MAE:    {cv_mae:.4f}")
print(f"Part 2 - Optimal Budget B*:       {optimal_budget}")
print(f"Part 2 - Budgeted CV-MAE:         {optimal_cv_mae:.4f}")
print(f"Part 3 - Coverage Rate:           {overall_coverage:.4f}")
print(f"Part 3 - MSIS Score:              {overall_msis:.4f}")
print(f"{'='*60}")

PART 3: PREDICTION INTERVAL CONSTRUCTION & EVALUATION
Using optimal budget B* = 50
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw

Processing Fold 1/5
Step 1: Computing residuals and variance modeling...
Step 2: Laplace-based interval simulation...
Fold 1 Coverage Rate: 0.5688
Fold 1 Mean MSIS: 0.1875

Processing Fold 2/5
Step 1: Computing residuals and variance modeling...
Step 2: Laplace-based interval simulation...
Fold 2 Coverage Rate: 0.5535
Fold 2 Mean MSIS: 0.2047

Processing Fold 3/5
Step 1: Computing residuals and variance modeling...
Step 2: Laplace-based interval simulation...
Fold 3 Coverage Rate: 0.5657
Fold 3 Mean MSIS: 0.1732

Processing Fold 4/5
Step 1: Computing residuals and variance modeling...
Step 2: Laplace-based interval simulation...
Fold 4 Coverage Rate: 0.5528
Fold 4 Mean MSIS: 0.2168

Processing Fold 5/5
Step 1: Computing resi

In [ ]:
# Define L1 Budget Levels
budget_levels = [70, 100]
print("Budget levels (Bi):", budget_levels)

budgeted_cv_results = {}

# Setup Gurobi environment
env = Env(empty=True)
env.setParam("WLSACCESSID", "af427699-3d84-4be7-8375-848e1c01fe5d")
env.setParam("WLSSECRET", "660f7b8c-a673-4870-94ae-c0ad37c28454")
env.setParam("LICENSEID", 2678165)
env.start()


def create_hinge_features(X_data, knots, feature_names, quantiles):
    X_hinge = X_data.copy()
    for feature in feature_names:
        for i, q in enumerate(quantiles):
            knot_val = knots[feature][i]
            hinge_name = f"{feature}_hinge_q{int(q * 100)}"
            X_hinge[hinge_name] = np.maximum(0, X_data[feature] - knot_val)
    X_hinge['intercept'] = 1
    return X_hinge

try:
    for budget in budget_levels:
        print(f"\n{'=' * 50}\nProcessing Budget Level: {budget}\n{'=' * 50}")

        kf = KFold(n_splits=5)
        mae_scores_budget = []

        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            print(f"\nFold {fold + 1}/5 for Budget {budget}")

            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            quantiles = [0.10, 0.25, 0.5, 0.75, 0.9]
            knots = {
                feature: [np.percentile(X_train[feature], q * 100) for q in quantiles]
                for feature in feature_names
            }

            X_train_hinge = create_hinge_features(X_train, knots, feature_names, quantiles)
            X_test_hinge = create_hinge_features(X_test, knots, feature_names, quantiles)

            n_samples, n_features = X_train_hinge.shape

            model = Model(f"BudgetedSpline_B{budget}_F{fold + 1}", env=env)
            model.setParam('OutputFlag', 0)

            beta = model.addVars(n_features, lb=-GRB.INFINITY, name="beta")
            pred_y = model.addVars(n_samples, name="pred_y")
            abs_error = model.addVars(n_samples, name="abs_error")
            beta_pos = model.addVars(n_features, name="beta_pos")
            beta_neg = model.addVars(n_features, name="beta_neg")

            model.setObjective(quicksum(abs_error[i] for i in range(n_samples)), GRB.MINIMIZE)

            for i in range(n_samples):
                model.addConstr(pred_y[i] == quicksum(beta[j] * X_train_hinge.iloc[i, j] for j in range(n_features)))
                model.addConstr(abs_error[i] >= y_train.iloc[i] - pred_y[i])
                model.addConstr(abs_error[i] >= pred_y[i] - y_train.iloc[i])

            for j in range(n_features):
                model.addConstr(beta[j] == beta_pos[j] - beta_neg[j])

            model.addConstr(quicksum(beta_pos[j] + beta_neg[j] for j in range(n_features)) <= budget)
            model.optimize()

            if model.status == GRB.OPTIMAL:
                coefficients = np.array([beta[j].X for j in range(n_features)])
                y_pred = X_test_hinge.values @ coefficients
                mae = mean_absolute_error(y_test, y_pred)
                mae_scores_budget.append(mae)
                print(f"  Fold {fold + 1} MAE: {mae:.4f}")

                if fold == 0:
                    budgeted_cv_results[budget] = {
                        'coefficients': coefficients,
                        'feature_names': list(X_train_hinge.columns),
                        'l1_norm': np.sum(np.abs(coefficients))
                    }
            else:
                print(f"  Fold {fold + 1}: No optimal solution")
                mae_scores_budget.append(np.inf)

            model.dispose()

        valid_scores = [score for score in mae_scores_budget if score != np.inf]
        if valid_scores:
            cv_mae_budget = np.mean(valid_scores)
            budgeted_cv_results[budget]['cv_mae'] = cv_mae_budget
            print(f"Budget {budget} CV-MAE: {cv_mae_budget:.4f}")
        else:
            budgeted_cv_results[budget] = {'cv_mae': np.inf}
            print(f"Budget {budget}: All folds failed")

    # Model Comparison
    print("\nMODEL COMPARISON RESULTS\n" + "=" * 60)
    valid_budgets = {b: r['cv_mae'] for b, r in budgeted_cv_results.items() if r['cv_mae'] != np.inf}

    if valid_budgets:
        optimal_budget = min(valid_budgets, key=valid_budgets.get)
        optimal_cv_mae = valid_budgets[optimal_budget]

        print(f"\nOptimal Budget B* = {optimal_budget}")
        print(f"Optimal CV-MAE = {optimal_cv_mae:.4f}")

        # Refit using full data
        quantiles = [0.10, 0.25, 0.5, 0.75, 0.9]
        knots = {
            feature: [np.percentile(X[feature], q * 100) for q in quantiles]
            for feature in feature_names
        }
        X_full_hinge = create_hinge_features(X, knots, feature_names, quantiles)
        n_samples_full, n_features_full = X_full_hinge.shape

        model_final = Model("RefitFinalModel", env=env)
        model_final.setParam('OutputFlag', 0)

        beta_final = model_final.addVars(n_features_full, lb=-GRB.INFINITY, name="beta")
        pred_y_final = model_final.addVars(n_samples_full, name="pred_y")
        abs_error_final = model_final.addVars(n_samples_full, name="abs_error")
        beta_pos_final = model_final.addVars(n_features_full, name="beta_pos")
        beta_neg_final = model_final.addVars(n_features_full, name="beta_neg")

        model_final.setObjective(quicksum(abs_error_final[i] for i in range(n_samples_full)), GRB.MINIMIZE)

        for i in range(n_samples_full):
            model_final.addConstr(pred_y_final[i] == quicksum(beta_final[j] * X_full_hinge.iloc[i, j] for j in range(n_features_full)))
            model_final.addConstr(abs_error_final[i] >= y.iloc[i] - pred_y_final[i])
            model_final.addConstr(abs_error_final[i] >= pred_y_final[i] - y.iloc[i])

        for j in range(n_features_full):
            model_final.addConstr(beta_final[j] == beta_pos_final[j] - beta_neg_final[j])

        model_final.addConstr(quicksum(beta_pos_final[j] + beta_neg_final[j] for j in range(n_features_full)) <= optimal_budget)
        model_final.optimize()

        if model_final.status == GRB.OPTIMAL:
            final_coefficients = np.array([beta_final[j].X for j in range(n_features_full)])
            feature_names_full = list(X_full_hinge.columns)

            nonzero_count = np.sum(np.abs(final_coefficients) > 0.001)
            l1_norm_final = np.sum(np.abs(final_coefficients))

            print(f"\nNumber of nonzero coefficients (|\u03b2| > 0.001): {nonzero_count}")
            print(f"L1 norm of coefficients: {l1_norm_final:.4f}")

            gamma_coeffs = [
                (abs(coeff), name, coeff)
                for coeff, name in zip(final_coefficients, feature_names_full)
                if 'hinge' in name and abs(coeff) > 0.001
            ]

            if gamma_coeffs:
                gamma_abs_vals = [g[0] for g in gamma_coeffs]
                print(f"\nEffect of spline coefficients (estimated gammas):")
                print(f"  Average |\u03b3|: {np.mean(gamma_abs_vals):.4f}")
                print(f"  Max |\u03b3|: {np.max(gamma_abs_vals):.4f}")
                print(f"  Number of active \u03b3 terms: {len(gamma_coeffs)}")

                gamma_coeffs.sort(reverse=True)
                print(f"\nTop 10 \u03b3 coefficients:")
                for i, (abs_val, name, coeff) in enumerate(gamma_coeffs[:10]):
                    print(f"  {i + 1:2d}. {name:25s}: {coeff:8.4f} (|{abs_val:.4f}|)")
            else:
                print("  No active \u03b3 coefficients found.")
        else:
            print("Refit model failed.")

finally:
    env.dispose()

Budget levels (Bi): [70, 100]
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2678165
Academic license 2678165 - for non-commercial use only - registered to 11___@g.nccu.edu.tw

Processing Budget Level: 70

Fold 1/5 for Budget 70
  Fold 1 MAE: 1.4100

Fold 2/5 for Budget 70
  Fold 2 MAE: 1.3916

Fold 3/5 for Budget 70
  Fold 3 MAE: 1.3341

Fold 4/5 for Budget 70
  Fold 4 MAE: 1.3392

Fold 5/5 for Budget 70
  Fold 5 MAE: 1.2697
Budget 70 CV-MAE: 1.3489

Processing Budget Level: 100

Fold 1/5 for Budget 100
  Fold 1 MAE: 1.4100

Fold 2/5 for Budget 100
  Fold 2 MAE: 1.3916

Fold 3/5 for Budget 100
  Fold 3 MAE: 1.3341

Fold 4/5 for Budget 100
  Fold 4 MAE: 1.3392

Fold 5/5 for Budget 100
  Fold 5 MAE: 1.2697
Budget 100 CV-MAE: 1.3489

MODEL COMPARISON RESULTS

Optimal Budget B* = 70
Optimal CV-MAE = 1.3489

Number of nonzero coefficients (|β| > 0.001): 29
L1 norm of coefficients: 44.6751

Effect of spline coefficients (estimated gammas):
  Average |γ|: 